# Assignment 4: Explainability

*Part of the course:
Machine Learning (code: INFOB3ML), fall 2023, Utrecht University*

Total points: 10 (100%)

Deadline: Friday 3 November, 23:59

**Write your names and student numbers here: ___**

Submit one ipynb file per pair.

**Before you submit, click Kernel > Restart & Run All to make sure you submit a working version of your code!**



## Installation

For this assignment, we are going to use the following Python packages:

matplotlib, pandas, statsmodels, interpret, scikit-learn, openpyxl and graphviz

In [709]:
# Installing packages
# !pip install graphviz
# !pip install matplotlib pandas numpy statsmodels interpret scikit-learn openpyxl

## Downloading the data
We are going to use the combined cycle power plant dataset. This dataset contains 9568 data points collected from a Combined Cycle Power Plant over 6 years (2006-2011), when the power plant was set to work with full load. We have the following features: hourly average ambient variables Temperature (T), Ambient Pressure (AP), Relative Humidity (RH) and Exhaust Vacuum (V). We will train ML models to predict the net hourly electrical energy output (EP) of the plant.

For a detailed description, see: [[Description](https://archive.ics.uci.edu/ml/datasets/combined+cycle+power+plant)]

We first need to download and prepare data. 


In [710]:
# Download and unzip data

# Windows users: download the data from https://archive.ics.uci.edu/ml/machine-learning-databases/00294/CCPP.zip
# and unzip the file manually in the same folder as the python notebook

# Note: this cell works fine on Linux based systems and Google Colab
# If you run it on a Windows machine, you will get an error (...'wget' is not recognized as an internal or external command...)
# !wget -c https://archive.ics.uci.edu/ml/machine-learning-databases/00294/CCPP.zip
# !unzip CCPP.zip

## Loading and preprocessing the data
We split the data into training (first 5000 instances) and validation (the subsequent 2000) and test (the last 2568) sets. We will use the training set to train a model, and validation set to optimize the model hyper-parameters. 


In [711]:
# Load and prepare data
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# global variables
DATA_FILENAME = 'CCPP/Folds5x2_pp.xlsx'
FEATURE_NAMES = ['AT', 'V', 'AP', 'RH']
LABEL_NAME = 'PE'
# Load the data from the excel file
def load_data():
    def split_feature_label(data_set):
        features = data_set[FEATURE_NAMES]
        labels = data_set[LABEL_NAME]
        return features, labels

    data = pd.read_excel(DATA_FILENAME)
    train_set, dev_set, test_set = data[:5000], data[5000: 7000], data[7000:]

    train_features, train_labels = split_feature_label(train_set)
    dev_features, dev_labels = split_feature_label(dev_set)
    test_features, test_labels = split_feature_label(test_set)

    return train_features, train_labels, dev_features, \
        dev_labels, test_features, test_labels


# preprocess (by z-normalization) the data for the regression task
# return the normalized feature sets and corresponding target variables 
def prepare_load_regression_data():
    train_features, train_labels, dev_features, \
        dev_labels, test_features, test_labels = load_data()

    scaler = StandardScaler()
    scaler = scaler.fit(train_features)

    print(train_features)

    train_features = pd.DataFrame(data=scaler.transform(train_features), columns=FEATURE_NAMES)
    dev_features = pd.DataFrame(data=scaler.transform(dev_features), columns=FEATURE_NAMES)
    test_features = pd.DataFrame(data=scaler.transform(test_features), columns=FEATURE_NAMES)

    return train_features, train_labels, dev_features, \
        dev_labels, test_features, test_labels, scaler



## Training and Interpreting a Linear Regression Model

**Q1**. (10)% Train a linear regression model (we recommend the statsmodels package) and report $R^2$ (goodness of fit) statistic.

For model interpretability, provide for each feature (+ the bias variable) the following in tabular format: 
* Weight estimates
* SE (standard error of estimates) 
* T statistics

*Answer*
Our model has an $R^2$ of 0.929. Tabular data of weights, SE and T statistics can be found in the code cell below.

Further Questions regarding the linear model (answers to be included in the notebook): 

**Q2**. (10%) Which three features are the most important?

*Answer*
The most important features can be obtained by looking at the highest absolute value of the T-statistic (exluding our bias feature):
1. AT
2. RH
3. V

**Q3**. (10%) How does the gas turbine energy yield (EP) change with unit (one degree C) increase of the ambient temperature given that all other feature values remain the same? (Note: Here you should consider whether you use the original or z-normalized features to train your linear model.)

*Answer*
EP changes by $\approx$ -1.9999 with unit change of AT.

**Q4**. (10%) Show bar graph illustrations of the feature effects for the first two development set instances.

*Answer*
Bar plots and dev instances are printed in the code cell below.

In [ ]:
# We recommend the statsmodels package
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
# Hint, by default this sm does not include the bias/offset term w_0
# thus, you should add it yourself using sm.add_constant()

## Q1
# Linear regression
# Get the preprocessed data for linear regression modeling
train_features, train_labels, dev_features, dev_labels, test_features, test_labels, scaler = prepare_load_regression_data()

# Add w0 constant
train_features = sm.add_constant(train_features)
dev_features = sm.add_constant(dev_features)
test_features = sm.add_constant(test_features)

# Train model
mod = sm.OLS(train_labels, train_features, hasconst=True)
res = mod.fit()
print(res.summary())

## Q3
# Create unit change data
data = pd.DataFrame([[15, 40, 1000, 70], [16, 40, 1000, 70]], columns=FEATURE_NAMES)
unit_features = pd.DataFrame(scaler.transform(data), columns=FEATURE_NAMES)
unit_features = sm.add_constant(unit_features, has_constant='add')

# Predict target and assess change
pred = res.predict(unit_features)
print('\nQ3 \n Data:')
print(unit_features)
print('\nEP effect with unit change on AT')
print(pred)
print('difference: ')
print(pred[1]-pred[0])

## Q4
# Fetch data
dev_samples = dev_features.head(2)
weights = res.params

print('\nQ4 \n Data:')
print(dev_samples)
print(weights)

# Calculate feature effects
dev_results = dev_samples.apply(lambda r: r * weights, axis=1)

print('\nResults:')
print(dev_results)

# Plot
for idx, row in dev_results.iterrows():
    plt.barh(FEATURE_NAMES, row[1:])
    plt.show()


**Q5.** (10%) Reflection: why would training a regression tree not work well for this dataset in terms of model interpretability? (answer in the notebook)

- Continous, zelfs als je 3 discrete waardes zou kiezen, zo de de boom alsnog enorm groot worden om alles te kunnen coveren.


## Training and Interpreting an Explainable Boosting Model (Generalized Additive Model)
**Q6**. (10%) Train a Explainable Boosting Machine (with [interpret.ml](https://github.com/interpretml/interpret/))

(Note on grading: Training EBM 4%, global and local explanation visualizations - see below -  3% each)

For a tutorial see: [[Tutorial](https://nbviewer.org/github/interpretml/interpret/blob/master/examples/python/notebooks/Interpretable%20Regression%20Methods.ipynb)]


* (3%) Visualize/provide global (model-wise) feature importances for EBM as a table or figure. What are the most important two features in EBM? Are they the same as in the linear model?

* (3%) Visualize local (instance-wise) feature importances for a development set instance of your choice.

*Answer*
Global feature importance is provided in code cell below. The most important features are:
- AT
- V

This differs from the linear model, which suggested that RH was the second most important feature.

Local feature importances are also provided below. Instances can be interactively chosen in the graph.



In [ ]:
from interpret.glassbox import ExplainableBoostingRegressor
from interpret import show

# EBM
ebm = ExplainableBoostingRegressor()
ebm.fit(train_features, train_labels)

ebm_global = ebm.explain_global()
show(ebm_global)

ebm_local = ebm.explain_local(dev_features, dev_labels)
show(ebm_local)

## Training and Explaining Neural Networks
**Q7**. (10%) Train a Neural Network (using the training and validation sets): One-layer MLP (with ReLU activation function and 50 to 100 hidden neurons) 

We recommend to use the Adam optimizer. Fine-tune the learning rate and any other hyper-parameters you find necessary. 

For a tutorial see: [[Tutorial](https://scikit-learn.org/stable/modules/neural_networks_supervised.html)]

Your code should report the results following the instructions below:

Note on grading: training NN: 4%, answering below sub-questions 3% each.

* (3%) Apply the trained neural network model on the test set and report Root Mean Square Error (RMSE) performance measure.

* (3%) Analyzing factors influencing the neural network predictions. 
See the [Documentation](https://scikit-learn.org/stable/modules/partial_dependence.html) to use Partial Dependence Plot (PDP)  implementation in python. Use the trained one-layer MLP model to  Generate and report a bivariate PDP using 'AT' (Ambient Temperature) and 'V' (Exhaust Vacuum) features (Note: not two univariate PDPs but one bivariate PDP).


In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.inspection import PartialDependenceDisplay

# One-layer MLP : you can use  learning_rate_init=0.001 to get a reasonable model, optimize other parameters by experimentation
# We advice that you name variable for the mlp regressor model 'mlp_reg' so that it will be consistent 
# with the scripts to call your implementation of PFI later in Q8:
mlp_reg= # ... your scripts here and below

### Generating Model-Agnostic Global Explanations for NN predictions

**Permutation Feature Importance (PFI)**

**Q8**. (20%) Implement the permutation feature importance algorithm using RMSE as the error function (%12). No existing libraries (save measuring RMSE) are allowed to use. We will implement it ourselves.
- (%4) Visualize feature importances obtained by PFI for the NN (one-layer MLP) model you trained using a bar graph.
- (%4) Reflection: What are the most important two features obtained by PFI for MLP model? How do these two features compare to the top two features from the Linear Model and the EBM?

In [ ]:
# Use the below function signature and the parameters to implement PFI
def PFI(X, labels, model, rmse):
# Your scripts implementing PFI here




# Do not modify this part (unless you change the variable name mlp_reg above)
base_rmse = mean_squared_error(dev_labels, mlp_reg.predict(dev_features) , squared=False)
results_df = PFI(dev_features, dev_labels, mlp_reg, base_rmse)

In [ ]:
# Your scripts to visualize results_df using a bar graph here


### Generating Model-Agnostic Explanations for NN predictions
You can check the tutorials for LIME explanations for neural networks 
[[LIME Tutorial](https://nbviewer.org/github/interpretml/interpret/blob/develop/examples/python/Explaining_Blackbox_Regressors.ipynb)]


**Q9**. (10%) Provide LIME-based explanations for trained NN model's predictions on two randomly selected test set instances.  

In [ ]:
# Global explanations
import graphviz
from interpret import show

# Local explanations (LIME)
from interpret.blackbox import LimeTabular
